In [73]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter 
from sklearn.preprocessing import scale
from modAL.density import information_density
np.random.seed(42)
import math

In [74]:
'''
ft:挑选后的特征
t:特征对应金标准(无)
name:样本名称
c: 特征对应簇
'''
df1 = pd.read_csv('./feats_bank_train_1111.csv',header = None)
# print(df1.shape)
ft = df1.values #特征样本/128维
# ft = scale(ft)

df2 = pd.read_csv('./train_1111.txt',sep = ' ',header = None)
name = df2.iloc[:,0].values #图片位置名称
# t = df2.iloc[:,1].values #金标准0/1
# print('Original dataset:', Counter(t))

In [75]:
ratio=0.50
total_num = len(name)
sel_num= int(np.floor(ratio*total_num))
sel_num

21965

In [76]:
from __future__ import print_function
import torch
from torch.autograd import Variable

def farthest_point_sample(xyz, npoint): 

    """
    Input:
        xyz: pointcloud data, [B, N, 3]
        npoint: number of samples
    Return:
        centroids: sampled pointcloud index, [B, npoint]
    """

#     xyz = xyz.transpose(2,1)
    device = xyz.device
    B, N, C = xyz.shape
    
    centroids = torch.zeros(B, npoint, dtype=torch.long).to(device)     # 采样点矩阵（B, npoint）
    distance = torch.ones(B, N).to(device) * 1e10                       # 采样点到所有点距离（B, N）

    batch_indices = torch.arange(B, dtype=torch.long).to(device)        # batch_size 数组

    #farthest = torch.randint(0, N, (B,), dtype=torch.long).to(device)  # 初始时随机选择一点
    
    barycenter = torch.sum((xyz), 1)                                    #计算重心坐标 及 距离重心最远的点
    barycenter = barycenter/xyz.shape[1]
    barycenter = barycenter.view(B, 1, C)

    dist = torch.sum((xyz - barycenter) ** 2, -1)
    farthest = torch.max(dist,1)[1]                                     #将距离重心最远的点作为第一个点

    for i in range(npoint):
#         print("-------------------------------------------------------")
#         print("The %d farthest pts %s " % (i, farthest))
        centroids[:, i] = farthest                                      # 更新第i个最远点
        centroid = xyz[batch_indices, farthest, :].view(B, 1, C)        # 取出这个最远点的xyz坐标
        dist = torch.sum((xyz - centroid) ** 2, -1)                     # 计算点集中的所有点到这个最远点的欧式距离
#         print("dist    : ", dist)
        mask = dist < distance
#         print("mask %i : %s" % (i,mask))
        distance[mask] = dist[mask]                                     # 更新distance，记录样本中每个点距离所有已出现的采样点的最小距离
#         print("distance: ", distance)

        farthest = torch.max(distance, -1)[1]                           # 返回最远点索引
 
    return centroids

# if __name__ == '__main__':

#     sim_data = Variable(torch.rand(2,10000,128))
#     print(sim_data)

#     centroids = farthest_point_sample(sim_data, 100)
    
#     print("Sampled pts: ", centroids)

In [77]:
'''
Maxcover
'''
import math

# index=[] #index[i]表示第i个聚类对应的index列表
# ft_tensor=[] #ft_part[i]表示第i个聚类对应的特征向量列表
new_index=[] #新列表

# ft_tensor.append(torch.Tensor(ft))
ft1 = ft[np.newaxis, :]
ft_tensor = torch.Tensor(ft1)

#     #注意：比例
# #     clf = IsolationForest(contamination=ratio,n_jobs=20).fit(ft_part[i])
    
# #     novel_pred = clf.predict(ft_part[i])
# #     li = np.where(novel_pred==-1)
    
# #     novel_scores = clf.score_samples(ft_part[i])
# #     li = np.argsort(novel_scores)[:sel_num]



centroids = farthest_point_sample(ft_tensor, sel_num).numpy().flatten()
# print("Sampled pts: ", centroids)
    
ft_sel = ft[centroids]

name_sel = name[centroids]
len(name_sel)

21965

In [78]:
'''
注意保存csv的后缀
'''
print(__doc__)
dataframe1 = pd.DataFrame({'image_name':name_sel})
dataframe1.to_csv("./Coreset_pick_1126/MAE_R50_Coreset.txt",index=False,sep=',',header = None)


注意保存csv的后缀

